<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Breast_Diagnosis_Model_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install tensorflow-addons

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define your data directory path and resolutions
data_dir = '/content/drive/My Drive/data2'
resolutions = [(30, 30), (70, 70), (100, 100), (150, 150), (250, 250)]

# Get class names
classes = [cls for cls in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, cls))]

# Function to load and preprocess data
def load_data(data_dir, resolutions, classes):
    data_dict = {res: {'X': [], 'y': []} for res in resolutions}
    for cls in classes:
        cls_dir = os.path.join(data_dir, cls)
        img_files = os.listdir(cls_dir)
        for img_file in img_files:
            img_path = os.path.join(cls_dir, img_file)
            img = Image.open(img_path)
            for res in resolutions:
                img_resized = img.resize(res)
                img_array = img_to_array(img_resized) / 255.0
                data_dict[res]['X'].append(img_array)
                data_dict[res]['y'].append(classes.index(cls))
    for res in resolutions:
        data_dict[res]['X'] = np.array(data_dict[res]['X'])
        data_dict[res]['y'] = np.array(data_dict[res]['y'])
    return data_dict

# Build and compile the model
def build_compile_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Plot confusion matrices
def plot_confusion_matrices(cm_train, cm_test, classes, filename):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    for i, (cm, title) in enumerate(zip([cm_train, cm_test], ['Train', 'Test'])):
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax[i])
        ax[i].set_title(f'{title} Confusion Matrix')
        ax[i].set_xlabel('Predicted label')
        ax[i].set_ylabel('True label')
        ax[i].set_xticklabels(classes, rotation=45)
        ax[i].set_yticklabels(classes)
    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

# Plot ROC curves
def plot_roc_curves(histories, X_test_dict, y_test_dict, classes, resolutions):
    plt.figure(figsize=(10, 8))
    for i, resolution in enumerate(resolutions):
        X_test = X_test_dict[resolution]
        y_test = y_test_dict[resolution]
        y_prob = histories[i].model.predict(X_test)
        for j, cls in enumerate(classes):
            fpr, tpr, _ = roc_curve(y_test == j, y_prob[:, j])
            plt.plot(fpr, tpr, lw=2, label=f'Class {cls} (Resolution {resolution})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Different Resolutions')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig('roc_curves.png')
    plt.show()

# Plot Boxplot
def plot_boxplot(metrics):
    melted_df = pd.melt(pd.DataFrame(metrics), id_vars='Resolution', var_name='Metric', value_name='Score')
    plt.figure(figsize=(10, 8))
    sns.boxplot(x='Resolution', y='Score', hue='Metric', data=melted_df)
    plt.title('Performance Metrics Boxplot')
    plt.savefig('boxplot_curves.png')
    plt.show()

# Load and preprocess data
data_dict = load_data(data_dir, resolutions, classes)

# Train models and collect metrics
histories, metrics = [], []
X_test_dict, y_test_dict = {}, {}

for res in resolutions:
    X = data_dict[res]['X']
    y = data_dict[res]['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    X_test_dict[res] = X_test
    y_test_dict[res] = y_test
    model = build_compile_model((res[0], res[1], 3), len(classes))
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=0)
    y_pred_train = np.argmax(model.predict(X_train), axis=1)
    y_pred_test = np.argmax(model.predict(X_test), axis=1)
    cm_train = confusion_matrix(y_train, y_pred_train)
    cm_test = confusion_matrix(y_test, y_pred_test)
    report = classification_report(y_test, y_pred_test, target_names=classes, output_dict=True)
    metrics.append({
        'Resolution': f'{res[0]}x{res[1]}',
        'Train Accuracy': history.history['accuracy'][-1],
        'Test Accuracy': history.history['val_accuracy'][-1],
        'Train Precision': report['macro avg']['precision'],
        'Test Precision': report['macro avg']['recall'],
        'Train Recall': report['macro avg']['recall'],
        'Test Recall': report['macro avg']['f1-score']
    })
    histories.append(history)

# Plot results
plot_confusion_matrices(cm_train, cm_test, classes, 'confusion_matrices.png')
plot_roc_curves(histories, X_test_dict, y_test_dict, classes, resolutions)
plot_boxplot(metrics)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
